# Gurobi Index Tracking Example (for comparison)

## Introduction

This index tracking example is meant to show the performance differences between Gurobi and TitanQ on a real life financial index tracking problem. We will be using the S&P 500 index as our benchmark.

More details for model formulation are found in the TitanQ based example of this problem.

In [ ]:
# Prompt the user and wait for an answer before installing dependencies.
# Note: Usage of a virtual Python environment is recommended.
if input("Install Python dependencies? [y/n]").lower() == "y":
    print("Installing dependencies (this can take a few minutes)...")
    !pip install -r "requirements.txt" --upgrade
else:
    print("Skipping dependency installation.")

#Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import time
#Gurobi specific libraries
import gurobipy as gp
from gurobipy import GRB
#Library to help with model generation
import model_generation

In [ ]:
#When should we model our time indices for? 

#Use Cached values for quicker access without downloading the data again
#Note: Downloading data from yFinance can be slow and buggy, recommend using a local cache
cache = True

#Pick an index which we would like to track
#SP500
ind_symbol = "^GSPC"
#FTSE
# ind_symbol = "^FTSE"
#CAC40
# ind_symbol = "^FCHI"

if cache:
    print("Using Cached Stock Data")
    stock_data, stock_returns, index_data, index_returns = model_generation.load_cache("2021", ind_symbol)
else:
    print("Downloading Stock Data, WARNING: This may take a while, and may error!")
    start_date = "2021-01-01"
    end_date = "2022-01-01"
    stock_data_dl, stock_returns_dl, index_data_dl, index_returns_dl = model_generation.download_data(index_name=ind_symbol, 
                                                                                          start_date=start_date, 
                                                                                          end_date=end_date)

In [ ]:
#Parameter max_invest: Maximum stocks of a single type to invest in
max_invest = 16

#Parameter min_budget: Minimum amount of money you'd like to spend
min_budget = 0

#Parameter max_budget: Maximum amount of money you're willing to spend
max_budget = 50000

#Parameter var_diff: Maximum difference in variance between the portfolio and the benchmark
var_diff = 0.1

In [ ]:
N = len(stock_returns.columns)  #Number of stocks
T = len(stock_returns)  #Number of time periods
print(f"Number of Stocks: {N}")
print(f"Number of Time Periods: {T}")

#Create a new Gurobi model
model = gp.Model("index_tracking")

print("Creating Variables...")
#Variables
x = model.addMVar(N, lb=0, ub=(max_invest-1), vtype=GRB.CONTINUOUS, name="x")  #Portfolio weights

print("Setting Objective...")


#Objective function: minimize tracking error
#Utility function to make this easier to read
W, b, offset = model_generation.get_objective(stock_returns=stock_returns, index_returns=index_returns, max_invest=max_invest)


model.setObjective(0.5 * (x @ W @ x) + b @ x + offset, GRB.MINIMIZE)
#Constraints
print("Adding Constraints...")

stock_init_price = stock_data.bfill().iloc[0, :].array
#Sum of portfolio weights is more than minimium budget
model.addConstr(sum(stock_init_price[i] * x[i] for i in range(N)) >= min_budget, "min_budget")
#Sum of portfolio weights is less than budget
model.addConstr(sum(stock_init_price[i] * x[i] for i in range(N)) <= max_budget, "max_budget")


#This is the variance of the portfolio
stock_cov = stock_returns.cov()
Q = stock_cov.values.astype(np.float32)
portfolio_variance = (x) @ (Q) @ (x)

#Variance bounds
target_variance = index_returns.var()
c_l = (1 - var_diff) * target_variance * (max_invest**2)
c_u = (1 + var_diff) * target_variance * (max_invest**2)

#Variance constraints
model.addConstr(portfolio_variance >= c_l, name='VarianceLowerBound')
model.addConstr(portfolio_variance <= c_u, name='VarianceUpperBound')

print("Model creation completed!")

In [ ]:
print("Solving the model!")
#Optimize the model
t0 = time.time()
model.optimize()
t1 = time.time()

In [ ]:
best_solution, best_obj = model_generation.analyze_results_gurobi(model, x, stock_init_price, W, b, offset)

In [ ]:
portfolio_returns_back = model_generation.calc_returns(stock_returns, best_solution.round())

tot_portfolio_ret = [np.sum(portfolio_returns_back[:x]) for x in range(T)]
tot_index_ret = [np.sum(index_returns[:x]) for x in range(T)]

plt.title(f"Index Return ({ind_symbol}) vs. Optimized Portfolio Performance")
plt.plot(stock_returns.index, tot_portfolio_ret, label="portfolio")
plt.plot(stock_returns.index, tot_index_ret, label=f"Index: {ind_symbol}")
plt.xlabel("Stock Trading Day")
plt.tick_params(axis='x', rotation=45)
plt.ylabel("Normalized Return")
plt.legend()

In [ ]:
#Fetching data for the forward time period (to test the model's performance on unseen data)
#Same as above, don't download the data unless necessary
if cache:
    stock_data_forward, stock_returns_forward, index_data_forward, index_returns_forward = model_generation.load_cache("2022", ind_symbol)
else:
    start_date = "2022-01-01"
    end_date = "2022-04-01"
    stock_data_forward, stock_returns_forward, index_data_forward, index_returns_forward = model_generation.download_data(index_name=ind_symbol, start_date=start_date, end_date=end_date)

In [ ]:
portfolio_returns_forward = model_generation.calc_returns(stock_returns_forward, best_solution)


tot_portfolio_ret = [np.sum(portfolio_returns_forward[:x]) for x in range(len(portfolio_returns_forward))]
tot_index_ret = [np.sum(index_returns_forward[:x]) for x in range(len(index_returns_forward))]

plt.title(f"Index Return ({ind_symbol}) vs. Optimized Portfolio Performance \n Forward in time")
plt.plot(stock_returns_forward.index, tot_portfolio_ret, label="portfolio")
plt.plot(stock_returns_forward.index, tot_index_ret, label=f"Index: {ind_symbol}")
plt.xlabel("Stock Trading Day")
plt.tick_params(axis='x', rotation=45)
plt.ylabel("Normalized Return")
plt.legend()